# modeling - feature selection

在我們進入多變量的建模預測前，必須要針對特徵進行必要的篩選  
確實，在概念上我們可以一股腦地把全部變數倒入nn模型訓練，模型會根據有用的變數去推論  
但這個方法有她麻煩的地方，訓練資料在特徵上變多其實對訓練時長也會有影響  
提前做過一輪特徵的選擇在訓練上的效率是比較好的  

# Lib

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import time
import sys

# 將 hwttp 資料夾加入 Python 搜尋路徑
sys.path.append(os.path.abspath('..'))
import hwttp.hwtoolkit as tk

# extra
import importlib

In [2]:
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import itertools

In [3]:
from pathlib import Path

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

# Load data

因為前面已經先建立過完整的dataset(避免從最開始load, transform資料這塊的時間，很慢)  
這邊可以直接load parquet  

In [3]:
data_paths = {'etag_5n_loc': '../data/cleaned/etag_5n_loc.csv',
              'section_info': '../data/cleaned/section_info.csv',
              'hw5_m04a_df': '../data/cleaned/hw5_m04a.csv',
              'congestion_table': '../data/cleaned/congestion_table.csv',
              'calendar_event': '../data/cleaned/calendar_event.csv',
              'road_build_event': '../data/cleaned/202301_10_road_build_event.xlsx',
              'traffic_accident_data': '../data/cleaned/202301_10_traffic_accident_data.xlsx'
             }

rs = tk.hw_df_resource(data_paths)
rs.load_raw_environment_info()
rs.load_raw_event_info()
rs.generate_mile_location_info()

Complete loading environment and gantry info
Complete loading raw event info
Complete generating mile location info


In [4]:
# 移除推測與交通落差相對大的節日
drop_event_list = ['婦女節', '愚人節', '復活節', '地球日', '感恩節', 'Black Friday', 'Cyber Monday', '冬至']
rs.calendar_event = rs.calendar_event[~rs.calendar_event.event_name.isin(drop_event_list)].copy()

In [5]:
# 旅行時間的資料
hw5_15watt = pd.read_csv("../data/features/hw5_15watt.csv")
hw5_15watt = hw5_15watt[~hw5_15watt.gf_gt.isin(['03F0150N-03F0140N', 
                                                '03F0201S-03A0041N', 
                                                '03F0201S-03F0217S'])].copy()

hw5_15watt = hw5_15watt[hw5_15watt.TimeStamp<'2024-01-01 00:00:00'].copy()
hw5_15watt['TimeStamp'] = pd.to_datetime(hw5_15watt['TimeStamp'])

In [6]:
hw5_15watt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 280320 entries, 131328 to 472799
Data columns (total 6 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   gf_gt                  280320 non-null  object        
 1   GantryFrom             280320 non-null  object        
 2   GantryTo               280320 non-null  object        
 3   TimeStamp              280320 non-null  datetime64[ns]
 4   WeightedAvgTravelTime  280320 non-null  float64       
 5   TotalTraffic           280320 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 15.0+ MB


# Combinations

我們可以透過定義不同函數的處理方式來做到增加變數類型  
這邊會把所有的變數類型塞進去，像是  
c: 路段擁擠時間資訊  
h: 節假日資訊  
t: 交通事故資訊  
r: 道路施工資訊  
p: 對應路段下游前五分鐘的旅行時間  
  
目標產出每個門架間旅行時間預設會參考到的全部特徵  

In [7]:
# 定義每個資料來源的特徵附加到既有df的function
def add_c(df):
    output_df = tk.add_congestion_condition(df, rs.congestion_table, rs.milelocation_info_df).copy()
    return output_df

def add_h(df):
    output_df = tk.add_calendar_event(df, rs.calendar_event).copy()
    return output_df

def add_t(df):
    *_, output_df = tk.add_traffic_event(df, rs.traffic_accident_data, rs.milelocation_info_df)
    return output_df

def add_r(df):
    *_, output_df = tk.add_road_build_event(df, rs.road_build_event, rs.milelocation_info_df)
    return output_df

def add_p(df):
    output_df = tk.add_ds_5prev_traveltime(df)
    return output_df
    
# function_set = [add_c, add_h, add_t, add_r, add_p]

# function放在dict中，可以在後面作法上直接呼叫
element_dict = {'_c': add_c, 
                '_h': add_h, 
                '_t': add_t, 
                '_r': add_r, 
                '_p': add_p
               }

In [8]:
# combinations，透過這個方式，我們預先定義對應門架中我希望附加上去的特徵種類
combinations = {'05F0001N-03F0150N': ['_c', '_h', '_t', '_r', '_p'],
                '05F0001N-03F0201S': ['_c', '_h', '_t', '_r', '_p'],
                '05F0055N-05F0001N': ['_c', '_h', '_t', '_r', '_p'],
                '05F0287N-05F0055N': ['_c', '_h', '_t', '_r', '_p'],
                '05F0309N-05F0287N': ['_c', '_h', '_t', '_r', '_p'],
                '05F0438N-05F0309N': ['_c', '_h', '_t', '_r', '_p'],
                '05F0438N-05FR143N': ['_c', '_h', '_t', '_r', '_p'],
                '05F0528N-05F0438N': ['_c', '_h', '_t', '_r', '_p'],
               }

In [11]:
# 結果儲存用results
results = dict()

# 執行所有排列组合
for gantry_pair, combo in tqdm(combinations.items()):
    
    # initialization
    result_df = hw5_15watt.copy()
    output_string = gantry_pair+'_b'
    print(gantry_pair, combo)
    combo_list = combo.copy()
    
    # 因為p的方法目前還沒有辦法處理多個欄位後再去接previous的問題，不做此處理會讓欄位分身暴增
    if '_p' in combo_list:
        combo_list.remove('_p')
        func = element_dict['_p']
        result_df = func(result_df)
        output_string += '_p'
    
    # 如果清單為空，要直接跳到最後不處理，最後沒有特別加這段處理
    print('combo_list = ', combo_list)
    
    for func_name in combo_list:
        func = element_dict[func_name]
        result_df = func(result_df)
        output_string += func_name

    # 最後才限縮gf_gt範圍，否則再找previous時會找不到資料
    result_df = result_df[result_df['gf_gt']==gantry_pair].copy()

    results[output_string] = result_df.copy()


  0%|          | 0/8 [00:00<?, ?it/s]

05F0001N-03F0150N ['_c', '_h', '_t', '_r', '_p']
combo_list =  ['_c', '_h', '_t', '_r']
locate same df.index before
locate same df.index before
locate same df.index before
locate same df.index before
locate same df.index before
locate same df.index before
locate same df.index before
locate same df.index before
locate same df.index before
locate same df.index before
locate same df.index before
locate same df.index before
locate same df.index before
locate same df.index before
locate same df.index before
locate same df.index before



100%|██████████| 179/179 [00:00<00:00, 1196.68it/s]


Complete checking traffic accident data, total event count = 115



100%|██████████| 115/115 [00:03<00:00, 29.86it/s]


...

Complete traffic accident data insertion
will return 2 object: target_traffic_accident_data, df



100%|██████████| 4354/4354 [00:08<00:00, 537.36it/s]


Complete checking & extract road_build_event



100%|██████████| 8/8 [16:05<00:00, 120.67s/it]

Complete road_build_event insertion to current df
will return 2 object: road_build_event, df


In [14]:
common_cols = ['gf_gt', 'GantryFrom', 'GantryTo', 'TimeStamp', 'WeightedAvgTravelTime']

for i in results.keys():
    print(i)
    print(f'number of extra features = {len(set(results[i].keys()) - set(common_cols))}')
    print('------------')

05F0001N-03F0150N_b_p_c_h_t_r
number of extra features = 58
------------
05F0001N-03F0201S_b_p_c_h_t_r
number of extra features = 58
------------
05F0055N-05F0001N_b_p_c_h_t_r
number of extra features = 58
------------
05F0287N-05F0055N_b_p_c_h_t_r
number of extra features = 58
------------
05F0309N-05F0287N_b_p_c_h_t_r
number of extra features = 58
------------
05F0438N-05F0309N_b_p_c_h_t_r
number of extra features = 58
------------
05F0438N-05FR143N_b_p_c_h_t_r
number of extra features = 58
------------
05F0528N-05F0438N_b_p_c_h_t_r
number of extra features = 58
------------


## to parquet

原始資料包含全部特徵部分，另存好方便下次快速載入使用  
parquet部分沒有object型別，處理過程需要轉換成string。

In [17]:
results['05F0001N-03F0150N_b_p_c_h_t_r'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35040 entries, 0 to 35039
Data columns (total 63 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   gf_gt                            35040 non-null  object        
 1   GantryFrom                       35040 non-null  object        
 2   GantryTo                         35040 non-null  object        
 3   TimeStamp                        35040 non-null  datetime64[ns]
 4   WeightedAvgTravelTime            35040 non-null  float64       
 5   TotalTraffic                     35040 non-null  float64       
 6   ds_prev_1_WATT                   35040 non-null  float64       
 7   ds_prev_2_WATT                   35040 non-null  float64       
 8   ds_prev_3_WATT                   35040 non-null  float64       
 9   ds_prev_4_WATT                   35040 non-null  float64       
 10  ds_prev_5_WATT                   35040 non-null  float64  

In [23]:
# fast save, next time no need to wait for long processing
for df_name, df in results.items():
    object_columns = df.select_dtypes(include='object').columns
    df[object_columns] = df[object_columns].astype('string')
    df.to_parquet(f'../data/features/all_features_by_intergrantry/{df_name}.parquet')

# Feature Selection

## read from parquet

In [5]:
parquet_path = "../data/features/all_features_by_intergrantry/"

p = Path(parquet_path)
results = dict()

for f in p.glob("*.parquet"):
    results[f.name.replace('.parquet', '')] = pd.read_parquet(f)

## main

這邊曾經嘗試使用RFECV來實作，但是該演算法在進行挑選特徵時會自動選擇好目標，不方便讓我們依據重要性做選擇。  
因此我選擇另一個方法來操作。  

In [6]:
def rolling_window_feature_selection(X, y, model, n_splits=5):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    feature_importances = np.zeros(X.shape[1])

    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        # print(f"Mean Squared Error: {mse}")

        # 累積特徵重要性
        feature_importances += model.feature_importances_

    # 計算平均特徵重要性
    feature_importances /= n_splits
    return feature_importances

In [ ]:
# 儲存feature selection結果用
restuls_fs = dict()

for item in tqdm(results.items()):
    cur_df_name = item[0]
    cur_df = item[1]

    if cur_df_name in results_fs.keys():
        continue
    
    # 分離特徵和目標變量
    X = cur_df.drop(columns=['WeightedAvgTravelTime', 'gf_gt', 'GantryFrom', 'GantryTo', 'TimeStamp', 'TotalTraffic'])
    y = cur_df['WeightedAvgTravelTime']
    
    # 定義基準模型
    base_model = RandomForestRegressor(n_estimators=100, random_state=42)
    
    # 計算特徵重要性
    feature_importances = rolling_window_feature_selection(X, y, base_model, n_splits=180)
    feature_importances_df = pd.DataFrame({'feature': X.columns,
                                           'importance': feature_importances
                                          }).sort_values(by='importance', ascending=False)
    
    # 選擇最重要的特徵
    selected_features = feature_importances_df['feature'].head(10).values  # 假設我們選擇前10個重要的特徵
    print(f"df = {cur_df_name} Selected features: {selected_features}")

    results_fs[cur_df_name] = feature_importances_df.copy()

print('feature selection complete')

 12%|█▎        | 1/8 [01:37<11:24, 97.74s/it]

df = 05F0001N-03F0150N_b_p_c_h_t_r Selected features: ['dayofweek' 'holiday_length' 'holiday_name_萬聖節' 'holiday_name_七夕情人節'
 'holiday_name_西洋情人節' 'holiday_continue' 'holiday_name_農曆新年'
 'holiday_name_國慶日' 'holiday_name_二二八紀念日' 'holiday_name_跨年元旦']


 25%|██▌       | 2/8 [03:10<09:28, 94.67s/it]

df = 05F0001N-03F0201S_b_p_c_h_t_r Selected features: ['dayofweek' 'holiday_length' 'holiday_continue' 'holiday_name_七夕情人節'
 'holiday_name_跨年元旦' 'holiday_name_西洋情人節' 'holiday_name_母親節'
 'holiday_name_中秋節' 'holiday_name_教師節' 'holiday_name_端午節']


 38%|███▊      | 3/8 [45:08<1:40:07, 1201.52s/it]

...

df = 05F0438N-05F0309N_b_p_c_h_t_r Selected features: ['ds_prev_1_WATT' 'ds_prev_5_WATT' 'ds_prev_2_WATT' 'ds_prev_4_WATT'
 'ds_prev_3_WATT' 'dayofweek' 'holiday_length' 'congestion_syndrome'
 'holiday_name_端午節' 'road_build']


 88%|████████▊ | 7/8 [3:07:33<27:52, 1672.19s/it]  

df = 05F0438N-05FR143N_b_p_c_h_t_r Selected features: ['road_build' 'dayofweek' 'holiday_length' 'road_block_count'
 'holiday_name_農曆新年' 'holiday_name_跨年元旦' 'congestion_syndrome'
 'holiday_continue' 'total_block_count' 'holiday_name_端午節']


100%|██████████| 8/8 [3:57:08<00:00, 1778.59s/it]

df = 05F0528N-05F0438N_b_p_c_h_t_r Selected features: ['ds_prev_1_WATT' 'ds_prev_3_WATT' 'ds_prev_2_WATT' 'ds_prev_4_WATT'
 'ds_prev_5_WATT' 'dayofweek' 'holiday_length' 'holiday_continue'
 'road_build' 'road_block_count']
feature selection complete


## quick view of results

透過特徵篩選過程我們計算過了平均的重要性，以下的語法可以快速看重要性>0的排序。  
會發現不同路段的特徵重要性不同，有的偏重參考下游的旅行時間，有的是節假日，還有部分靠前的是道路是否處於施工狀態。

In [ ]:
for i in results_fs.keys():
    print(i)
    display(results_fs[i].query('importance > 0'))

05F0001N-03F0150N_b_p_c_h_t_r


,feature,importance
8,dayofweek,0.610277
7,holiday_length,0.132508
23,holiday_name_萬聖節,0.090272
9,holiday_name_七夕情人節,0.035597
24,holiday_name_西洋情人節,0.028306
6,holiday_continue,0.028191
26,holiday_name_農曆新年,0.014103
16,holiday_name_國慶日,0.011962
12,holiday_name_二二八紀念日,0.011498
25,holiday_name_跨年元旦,0.008412


05F0001N-03F0201S_b_p_c_h_t_r


,feature,importance
8,dayofweek,0.636347
7,holiday_length,0.207830
6,holiday_continue,0.122040
9,holiday_name_七夕情人節,0.009345
25,holiday_name_跨年元旦,0.007429
24,holiday_name_西洋情人節,0.002793
18,holiday_name_母親節,0.002370
11,holiday_name_中秋節,0.002101
17,holiday_name_教師節,0.001563
21,holiday_name_端午節,0.001469


05F0055N-05F0001N_b_p_c_h_t_r


...

,feature,importance
0,ds_prev_1_WATT,7.595523e-01
4,ds_prev_5_WATT,5.281273e-02
1,ds_prev_2_WATT,5.242279e-02
3,ds_prev_4_WATT,3.896254e-02
2,ds_prev_3_WATT,3.596455e-02
8,dayofweek,2.365059e-02
7,holiday_length,1.144466e-02
5,congestion_syndrome,4.481054e-03
21,holiday_name_端午節,3.131000e-03
53,road_build,2.596335e-03


05F0438N-05FR143N_b_p_c_h_t_r


,feature,importance
53,road_build,0.419806
8,dayofweek,0.376563
7,holiday_length,0.055563
55,road_block_count,0.027151
26,holiday_name_農曆新年,0.017800
25,holiday_name_跨年元旦,0.017045
5,congestion_syndrome,0.016521
6,holiday_continue,0.016115
54,total_block_count,0.009573
21,holiday_name_端午節,0.009007


05F0528N-05F0438N_b_p_c_h_t_r


,feature,importance
0,ds_prev_1_WATT,2.186123e-01
2,ds_prev_3_WATT,1.770015e-01
1,ds_prev_2_WATT,1.741344e-01
3,ds_prev_4_WATT,1.697923e-01
4,ds_prev_5_WATT,1.678288e-01
8,dayofweek,4.713649e-02
7,holiday_length,1.368403e-02
6,holiday_continue,5.498046e-03
53,road_build,4.712490e-03
55,road_block_count,3.497024e-03


## to_csv

In [15]:
for i in results_fs.keys():
    f_name = f'../data/features/feature_selection_results/{i}.csv'
    results_fs[i].to_csv(f_name, index=False)